In [1]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoModelForMaskedLM

from utils import *
from dataset import *
from preprocess import *
from wrapper import *
from models import *
from pipeline import PipelineGED

np.random.seed(1024)

# device = torch.device("cpu")
device = torch.device("cuda:0")
ntf()

Simbert不能正常使用，除非你安装：bert4keras、tensorflow ，为了安装快捷，没有默认安装.... No module named 'bert4keras'


Building prefix dict from d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\holaj\AppData\Local\Temp\jieba.u842d550ffb06c1e965781a2cb0144564.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.258 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [d:\Apps\Anaconda3\envs\general-torch\lib\site-packages\synonyms\data\words.vector.gz] ...


In [3]:
train_df = pd.read_csv('../data/data-org/train.csv', sep='\t', index_col='id')

np.random.seed(297)
(train_df.label == 0).sum() / (train_df.label == 1).sum()

0.3402547393364929

In [4]:
import jieba

def get_trunc_text(text):
    sentences = [s for s in text.split('。') if s]
    if len(sentences) > 1:
        prompt = jieba.cut(sentences[1], cut_all=False).__next__()
        return sentences[0] + '。' + prompt
    subsentences = [s for s in sentences[0].split('，') if s]
    if len(subsentences) > 1:
        n_kept = ceil(len(subsentences) / 3)
        kept_subsentences = '，'.join(subsentences[:n_kept])
        prompt = jieba.cut(subsentences[n_kept], cut_all=False).__next__()
        return kept_subsentences + '，' + prompt
    words = list(jieba.cut(subsentences[0], cut_all=False))
    return ''.join(words[:len(words)//3])


In [5]:
pos_data = train_df[train_df.label == 1]
pos_data.head(10)

pos_data_trunc = pos_data.copy(deep=True).rename(columns={'text':'full_text'})
pos_data_trunc['text'] = pos_data_trunc.full_text.map(get_trunc_text)

pos_ds = DatasetWithAuxiliaryEmbeddings(pos_data_trunc, model_name='bigscience/bloom-560m', device='cuda')
pos_ds.prepare_dataset()

**步骤**:
1. sample一些training data，保留原正负比例（7:3）
2. 把正样本开头feed进generative model，直到得到eos，这些归类为扩充负样本
3. 在原样本上和原样本+扩充负样本上分别训练模型，在unseen dev set上对比performance

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from tqdm.notebook import tqdm_notebook


tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")

device = torch.device('cuda')
if 'cuda' in device.type:
    model.cuda()

generated_sentences = []

p = 0.25
dataloader = DataLoader(pos_ds.dataset['train'].with_format('torch'), batch_size=4)

for batch in tqdm_notebook(dataloader):
    if np.random.random() > p:
        continue
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        output = model.generate(
            **inputs, 
            max_new_tokens=64,
            eos_token_id=420,  
            forced_eos_token_id=420, 
            pad_token_id=tokenizer.convert_tokens_to_ids('<pad>'),
            no_repeat_ngram_size=3, 
            repetition_penalty=1.2, 
            top_k=5, 
        )
        generated_sentences.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

  0%|          | 0/8440 [00:00<?, ?it/s]

In [8]:
del model
torch.cuda.empty_cache()

In [9]:
generated_sentences = [s for s in generated_sentences if len(s) < 64]

In [10]:
len(generated_sentences)

7496

## Testing

In [11]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoModelForMaskedLM

from utils import *
from dataset import *
from preprocess import *
from wrapper import *
from models import *
from pipeline import PipelineGED

np.random.seed(1024)

# device = torch.device("cpu")
device = torch.device("cuda:0")
ntf()

In [12]:
MODEL_SIZE = 'base'
MODEL_ARCH = 'macbert+bert-wwm'

np.random.seed(297)

train_df = pd.read_csv('../data/data-org/train.csv', sep='\t', index_col='id')
test_df = pd.read_csv('../data/data-org/test.csv', sep='\t', index_col='id')

oob_model_name = None
if MODEL_ARCH == 'ernie':
    model_name = 'nghuyong/ernie-gram-zh'
elif MODEL_ARCH == 'macbert':
    model_name = 'hfl/chinese-macbert-base' if MODEL_SIZE == 'base' else 'hfl/chinese-macbert-large'
elif MODEL_ARCH == 'roberta-word-based':
    model_name = 'uer/roberta-base-word-chinese-cluecorpussmall'
elif MODEL_ARCH == 'macbert+bert-wwm':
    model_name = 'hfl/chinese-macbert-base'
    oob_model_name = 'KoichiYasuoka/chinese-bert-wwm-ext-upos'
else:
    raise NotImplementedError(f'Model {MODEL_ARCH} is not implemented yet.')


max_length = 64

test_dataset_config = {
    'model_name':model_name,
    'aux_model_name':oob_model_name,
    'maxlength':max_length,
    'train_val_split':-1,
    'test':True, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'device':device,
    'split_words':False, 
    'cut_all':False, 
}

test = DatasetWithAuxiliaryEmbeddings(df=test_df.reset_index(), **test_dataset_config)
test.tokenize()
test.construct_dataset()

train_dataset_config = {
    'model_name':model_name,
    'aux_model_name':oob_model_name,
    'maxlength':max_length,
    'train_val_split':-1,
    'test':False, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'device':device,
    'split_words':False, 
    'cut_all':False, 
}

train = DatasetWithAuxiliaryEmbeddings(df=train_df.reset_index(), **train_dataset_config)
train.tokenize()
train.construct_dataset()

ntf()

In [13]:
if MODEL_ARCH == 'macbert':
    checkpoints = [
        '../finetuned_models/ensemble/model0/checkpoint-2400/pytorch_model.bin',
    ]
elif MODEL_ARCH == 'ernie':
    checkpoints = [f'../finetuned_models/balanced_trial_ernie_gram/fold{i}/checkpoint-1910/pytorch_model.bin' for i in range(10)] 
elif MODEL_ARCH == 'roberta-word-based':
    checkpoints = [
        "../finetuned_models/word-based-roberta/model0/checkpoint-1500/pytorch_model.bin", 
    ]
elif MODEL_ARCH == 'macbert+bert-wwm':
    checkpoints = [
        "../finetuned_models/macbert_with_wwm_upos/model0/checkpoint-5832/pytorch_model.bin", 
        "../finetuned_models/macbert_with_wwm_upos/model1/checkpoint-6811/pytorch_model.bin", 
        "../finetuned_models/macbert_with_wwm_upos/model2/checkpoint-6776/pytorch_model.bin", 
        "../finetuned_models/macbert_with_wwm_upos/model3/checkpoint-5826/pytorch_model.bin", 
        "../finetuned_models/macbert_with_wwm_upos/model4/checkpoint-4860/pytorch_model.bin", 

    ]
else:
    print(f'Model {MODEL_ARCH} is not implemented yet.')

data_configs = {
    'model_name':model_name,
    'maxlength':max_length,
    'train_val_split':-1,
    'test':True, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'split_words':False, 
    'cut_all':False, 
}
clf = PipelineGED(
    model_name=model_name, 
    oob_model_name=oob_model_name, 
    data_configs=data_configs, 
    pooling_mode='cls',  
    model_architecture='bert_with_oob_model', 
)

def apply_ged_pipeline_oob(texts, checkpoints=checkpoints, majority_vote=False):
    probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True, display=False)
    return probs


Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at KoichiYasuoka/chinese-bert-wwm-ext-upos were not used when initializing Bert

In [14]:
texts = generated_sentences
probs = apply_ged_pipeline_oob(texts, checkpoints=checkpoints, )

100%|██████████| 469/469 [00:55<00:00,  8.49it/s]


In [15]:
generated_df = pd.DataFrame(data={'label':probs[0].argmax(1), 'text':texts})
generated_df

,label,text
0,0,为了营造和谐、安全的校园环境，确保广大未成年人的交通安全，天元区公安局交警大队在辖区范围内开...
1,0,在翻阅中国话剧100周年纪念活动资料时，他发现《红楼梦》这部戏的演出时间是1902年。
2,1,下午三点整，参加完会议后返回酒店。
3,0,"继亚马逊线下书店开张后,当当网也准备从线上走向线下，这一举动让不少读者感到意外。"
4,1,通过这次学习，使大家对党的十九届五中全会精神有了更深层次的理解和认识。
...,...,...
7491,0,代表们认为，政府只有不断改善民生，社会才能真正发展起来。
7492,0,近日，一张“重庆大学”学生在宿舍内与他人发生肢体冲突的照片在网上流传。
7493,0,我觉得这个答复和对这些问题的调查处理，都与我的工作有关。
7494,1,你叫的外卖送餐箱卫生吗？现实生活中好多叫外卖的老友都是直接从外卖哥的手中接过快餐，很少有人会...


In [16]:
train_df_expanded = pd.concat((train_df, generated_df))
(train_df_expanded.label == 0).sum() / len(train_df_expanded)

0.2982765485467266

In [17]:
(train_df.label == 0).sum() / len(train_df)

0.25387318496253897

In [18]:
train_df.to_csv('../data/data-gen/org/train.csv', sep='\t', index=False)
train_df_expanded.to_csv('../data/data-gen/expanded/train.csv', sep='\t', index=False)